# 自动生成工具使用示例


## 导入所需的包


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## 创建客户端

在本示例中，我们将使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 来访问 LLM。

`model` 被定义为 `gpt-4o-mini`。尝试将模型更改为 GitHub Models 市场中提供的其他模型，看看不同的结果。

作为一个快速测试，我们将运行一个简单的提示——`法国的首都是哪里`。


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 定义函数

在这个示例中，我们将为代理提供一个工具，该工具是一个包含可用度假目的地及其可用性的函数。

你可以将其想象为一个场景，例如旅行代理可以访问一个旅游数据库。

在浏览这个示例时，可以尝试定义代理可以调用的新函数和工具。


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## 定义函数工具  
为了让代理将 `vacation_destinations` 用作 `FunctionTool`，我们需要将其定义为一个函数工具。  

我们还需要为该工具提供一个描述，这有助于代理识别该工具在用户请求的任务中是用来做什么的。  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## 定义代理

现在我们可以在下面的代码中创建代理。我们定义了 `system_message`，用于向代理提供指令，帮助用户寻找度假目的地。

我们还将 `reflect_on_tool_use` 参数设置为 true。这允许使用 LLM 来处理工具调用的响应，并以自然语言发送响应。

你也可以将该参数设置为 false 来观察两者的区别。


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## 运行代理

现在我们可以运行代理，初始用户消息是请求去东京旅行。

你可以更改这个城市目的地，看看代理如何响应该城市的可用性。


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保准确性，但请注意，自动翻译可能包含错误或不准确之处。应以原始语言的文档作为权威来源。对于关键信息，建议使用专业人工翻译。对于因使用本翻译而引起的任何误解或误读，我们概不负责。
